In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt

In [9]:
SP500_df = pd.read_csv("SP500_quarterly_data.csv")
WB_df = pd.read_csv("Warren_Buffett_final_df_with_ticker.csv")


tickers = SP500_df['Ticker'].unique().tolist()
print(len(tickers))

WB_tickers = WB_df['ticker'].unique().tolist()
print(WB_tickers)
print(len(WB_tickers))

candidate_tickers = list(set(tickers) - set(WB_tickers))
print(candidate_tickers)
print(len(candidate_tickers))

498
['AAPL', 'ALLY', 'AMZN', 'AON', 'AXP', 'BAC', 'BATRK', 'C', 'CB', 'CHTR', 'COF', 'CVX', 'DEO', 'DPZ', 'DVA', 'FND', 'FWONK', 'HEI.A', 'JEF', 'KHC', 'KO', 'KR', 'LEN.B', 'LILA', 'LILAK', 'LLYVA', 'LLYVK', 'LPX', 'LSXMA', 'LSXMK', 'MA', 'MCO', 'NU', 'NVR', 'OXY', 'POOL', 'SIRI', 'SPY', 'STZ', 'TMUS', 'ULTA', 'V', 'VOO', 'VRSN']
44
['HIG', 'SWKS', 'IBKR', 'LYB', 'AMAT', 'PAYX', 'MCD', 'TPR', 'KMI', 'WTW', 'VLTO', 'NKE', 'ALB', 'DRI', 'ABT', 'WMB', 'AVY', 'TAP', 'BKR', 'EXE', 'UBER', 'EIX', 'BIIB', 'VRSK', 'BKNG', 'EXC', 'EXPD', 'IT', 'ROP', 'UHS', 'PNW', 'STX', 'ES', 'XYZ', 'PLD', 'MS', 'AWK', 'BSX', 'BEN', 'TRGP', 'FI', 'MOS', 'ELV', 'ADSK', 'LMT', 'STE', 'IVZ', 'CFG', 'DOC', 'ALGN', 'DLTR', 'PWR', 'FSLR', 'FOX', 'HLT', 'UAL', 'LDOS', 'NEE', 'TER', 'AVGO', 'UPS', 'MTCH', 'CPB', 'DOW', 'PTC', 'SWK', 'ABBV', 'EQT', 'KEY', 'PHM', 'ALL', 'DHR', 'ACN', 'CLX', 'MDT', 'NOC', 'CF', 'DAY', 'DE', 'MPWR', 'L', 'AMD', 'FIS', 'ATO', 'AEP', 'T', 'AES', 'CNP', 'ETR', 'TSN', 'CHD', 'LHX', 'AMCR', 'I

In [10]:
# Data Preparation (Financials, Balance Sheets, Cashflow)

# wanna apply ffill to the data
def quarterly_financials(ticker):
    stock = yf.Ticker(ticker)
    fin = stock.quarterly_financials
    return fin

def quarterly_balance_sheet(ticker):
    stock = yf.Ticker(ticker)
    bal = stock.quarterly_balance_sheet
    return bal

def quarterly_cashflow(ticker):
    stock = yf.Ticker(ticker)
    cf = stock.quarterly_cashflow
    return cf


financials_dfs = []
for ticker in candidate_tickers:
    fin = quarterly_financials(ticker).T 
    fin.index = pd.MultiIndex.from_product([[ticker], fin.index], names=["ticker", "date"])
    financials_dfs.append(fin)

balance_sheets_dfs = []
for ticker in candidate_tickers:
    bal = quarterly_balance_sheet(ticker).T
    bal.index = pd.MultiIndex.from_product([[ticker], bal.index], names=["ticker", "date"])
    balance_sheets_dfs.append(bal)

cashflow_dfs = []
for ticker in candidate_tickers:
    cf = quarterly_cashflow(ticker).T
    cf.index = pd.MultiIndex.from_product([[ticker], cf.index], names=["ticker", "date"])
    cashflow_dfs.append(cf)


financials_df = pd.concat(financials_dfs)
balance_sheets_df = pd.concat(balance_sheets_dfs)
cashflow_df = pd.concat(cashflow_dfs)

financials_df = financials_df.groupby(level=0).ffill()
balance_sheets_df = balance_sheets_df.groupby(level=0).ffill()
cashflow_df = cashflow_df.groupby(level=0).ffill()

# financials_df.to_csv("quarterly_financials.csv")
# balance_sheets_df.to_csv("quarterly_balance_sheets.csv")
# cashflow_df.to_csv("quarterly_cashflow.csv")

print(financials_df)


                   Tax Effect Of Unusual Items  Tax Rate For Calcs  \
ticker date                                                          
HIG    2025-06-30                 0.000000e+00            0.201445   
       2025-03-31                 0.000000e+00            0.195402   
       2024-12-31                 0.000000e+00            0.205773   
       2024-09-30                -1.943277e+05            0.194328   
       2024-06-30                 0.000000e+00            0.190789   
...                                        ...                 ...   
ICE    2025-03-31                -7.680000e+06            0.240000   
       2024-12-31                -3.465193e+06            0.216575   
       2024-09-30                -9.250000e+06            0.250000   
       2024-06-30                -3.900000e+06            0.260000   
       2024-03-31                -3.900000e+06            0.260000   

                   Total Unusual Items  \
ticker date                              
HIG  

In [11]:
financials_df.to_csv("SP500_quarterly_financials.csv")
balance_sheets_df.to_csv("SP500_quarterly_balance_sheets.csv")
cashflow_df.to_csv("SP500_quarterly_cashflow.csv")


In [16]:
financials_df = pd.read_csv("SP500_quarterly_financials.csv", index_col=[0, 1])
balance_sheets_df = pd.read_csv("SP500_quarterly_balance_sheets.csv", index_col=[0, 1])
cashflow_df = pd.read_csv("SP500_quarterly_cashflow.csv", index_col=[0, 1])

# Profitability: ROE, profit margins, operating margins
net_income = financials_df["Net Income"]
total_revenue = financials_df["Total Revenue"]
operating_income = financials_df["Operating Income"]
equity = balance_sheets_df["Common Stock Equity"]
total_assets = balance_sheets_df["Total Assets"]
gross_profit = financials_df["Gross Profit"]

roe = net_income / equity
profit_margin = net_income / total_revenue
operating_margin = operating_income / total_revenue
net_margin = net_income / total_revenue
roa = net_income / total_assets
gross_margins = gross_profit / total_revenue


profitability_df = pd.DataFrame({
    "ROE": roe,
    "Profit Margin": profit_margin,
    "Operating Margin": operating_margin,
    "Net Margin": net_margin,
    "ROA": roa,
    "Gross Margin": gross_margins
})

print(profitability_df)


                        ROE  Profit Margin  Operating Margin  Net Margin  \
ticker date                                                                
A      2024-04-30  0.049565       0.195804          0.230769    0.195804   
       2024-07-31  0.047772       0.178707          0.211027    0.178707   
       2024-10-31  0.059512       0.206349          0.239859    0.206349   
       2025-01-31  0.052763       0.189173          0.223676    0.189173   
       2025-04-30  0.035039       0.128897          0.179856    0.128897   
...                     ...            ...               ...         ...   
XYZ    2024-06-30  0.010116       0.031722          0.080963    0.031722   
       2024-09-30  0.014244       0.047484          0.086193    0.047484   
       2024-12-31  0.091500       0.322586          0.042707    0.322586   
       2025-03-31  0.008844       0.032897          0.086453    0.032897   
       2025-06-30  0.024302       0.088936          0.128564    0.088936   

           

In [17]:
# Growth - 
def revenue_growth(financials_df):
    revenue = financials_df["Total Revenue"]
    revenue = revenue.sort_index()
    return revenue.groupby(level=0).pct_change()

def earnings_growth(financials_df):
    earnings = financials_df["Net Income"]
    earnings = earnings.sort_index()
    return earnings.groupby(level=0).pct_change()

revenue_growth_df = revenue_growth(financials_df)
# print(revenue_growth_df)

earnings_growth_df = earnings_growth(financials_df)
# print(earnings_growth_df)


growth_df = pd.DataFrame({
    "Revenue Growth": revenue_growth_df,
    "Earnings Growth": earnings_growth_df
})

print(growth_df)

                   Revenue Growth  Earnings Growth
ticker date                                       
A      2024-04-30             NaN              NaN
       2024-07-31        0.003179        -0.084416
       2024-10-31        0.077947         0.244681
       2025-01-31       -0.011758        -0.094017
       2025-04-30       -0.007733        -0.323899
...                           ...              ...
XYZ    2024-06-30        0.000000         0.000000
       2024-09-30       -0.029203         0.453152
       2024-12-31        0.009498         5.858124
       2025-03-31       -0.043226        -0.902431
       2025-06-30        0.048973         1.835900

[2952 rows x 2 columns]


In [18]:
# Financial Health

debt_ratio = balance_sheets_df["Total Debt"] / balance_sheets_df["Total Assets"]
# print(debt_ratio)

debt_to_equity_ratio = balance_sheets_df["Total Debt"] / balance_sheets_df["Stockholders Equity"]
# print(debt_to_equity_ratio)

current_ratio = balance_sheets_df["Current Assets"] / balance_sheets_df["Current Liabilities"]
# print(current_ratio)

FCF = cashflow_df["Free Cash Flow"]
# print(FCF)

Financial_health_df = pd.DataFrame({
    "Debt Ratio": debt_ratio,
    "Debt to Equity Ratio": debt_to_equity_ratio,
    "Current Ratio": current_ratio,
    "Free Cash Flow": FCF
})

print(Financial_health_df)

                   Debt Ratio  Debt to Equity Ratio  Current Ratio  \
ticker date                                                          
A      2024-01-31         NaN                   NaN            NaN   
       2024-04-30    0.235446              0.411329       2.146578   
       2024-07-31    0.266642              0.496697       1.781499   
       2024-10-31    0.286173              0.574771       2.089182   
       2025-01-31    0.282273              0.557989       2.197432   
...                       ...                   ...            ...   
XYZ    2024-06-30    0.196264              0.381777       1.822569   
       2024-09-30    0.203799              0.371930       2.069222   
       2024-12-31    0.215311              0.372326       2.326041   
       2025-03-31    0.166090              0.281564       2.269257   
       2025-06-30    0.166108              0.276321       1.956811   

                   Free Cash Flow  
ticker date                        
A      2024-01-31

In [20]:
# Valuation

def get_info_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    return info

info_dfs = []
for ticker in candidate_tickers:
    info = get_info_data(ticker)
    info_df = pd.DataFrame([info])
    info_df["ticker"] = ticker
    info_dfs.append(info_df)

info_df = pd.concat(info_dfs, ignore_index=True)
# print(info_df)

# Trailing P/E Ratio
trailing_pe = info_df["trailingPE"]
# print(info_df[["ticker", "trailingPE"]])

# priceToBook
price_to_book = info_df["priceToBook"]
# print(info_df[["ticker", "priceToBook"]])

# enterpriseToEbitda (EV/EBITDA)
enterprise_to_ebitda = info_df["enterpriseToEbitda"]
# print(info_df[["ticker", "enterpriseToEbitda"]])

# enterpriseValue (EV)
enterprise_value = info_df["enterpriseValue"]
# print(info_df[["ticker", "enterpriseValue"]])

# marketCap
market_cap = info_df["marketCap"]
# print(info_df[["ticker", "marketCap"]])

# dividendYield
dividend_yield = info_df["dividendYield"]
# print(info_df[["ticker", "dividendYield"]])

fundamental_df = pd.DataFrame({
    "ticker": info_df["ticker"],
    "trailingPE": trailing_pe,
    "priceToBook": price_to_book,
    "enterpriseToEbitda": enterprise_to_ebitda,
    "enterpriseValue": enterprise_value,
    "marketCap": market_cap,
    "dividendYield": dividend_yield
})

print(fundamental_df)

    ticker  trailingPE  priceToBook  enterpriseToEbitda  enterpriseValue  \
0      HIG   12.017257     2.173399               8.433     3.809082e+10   
1     SWKS   29.856575     1.967446              12.704     1.101306e+10   
2     IBKR   32.757896     5.693378                 NaN    -4.714880e+10   
3      LYB  119.893616     1.521862              10.627     2.989289e+10   
4     AMAT   19.183770     6.566189              14.117     1.278162e+11   
..     ...         ...          ...                 ...              ...   
468    HII   20.238417     2.176541              13.447     1.319197e+10   
469    APO   25.463550     4.347812                 NaN     1.007542e+11   
470   SNPS   69.932790     9.442985              61.228     9.012095e+10   
471   PAYC   30.654520     6.898803              24.172     1.232769e+10   
472    ICE   33.896355     3.551391              19.896     1.198359e+11   

        marketCap  dividendYield  
0     37201866752           1.57  
1     11123194880

C:\Users\revan\AppData\Local\Temp\ipykernel_14960\152812651.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  info_df = pd.concat(info_dfs, ignore_index=True)


In [21]:
output_path = "SP500_Analysis.xlsx"

with pd.ExcelWriter(output_path) as writer:
    financials_df.to_excel(writer, sheet_name="Financials")
    balance_sheets_df.to_excel(writer, sheet_name="Balance_Sheets")
    cashflow_df.to_excel(writer, sheet_name="Cashflow")
    profitability_df.to_excel(writer, sheet_name="Profitability")
    growth_df.to_excel(writer, sheet_name="Growth")
    Financial_health_df.to_excel(writer, sheet_name="Financial_Health")
    fundamental_df.to_excel(writer, sheet_name="Valuation")

print(f"All DataFrames have been successfully written to '{output_path}'")

All DataFrames have been successfully written to 'SP500_Analysis.xlsx'
